## Analysis of Response in Time for a Third-Order Model

A synchronous machine connected to an infinite bus through transmission lines is being analyzed. The parameters considered are:

- **Stator Voltage (Vs):** 1.01 pu
- **Line Reactance (XL):** 0.1 pu
- **Mechanical Power (Pm):** from 0.5 pu to 1.0 pu
- **Infinite Bus Voltage (Eb):** 1.01 pu
- **Transient Time Constant for Direct-Axis (Td0′​):** from 0.01 s to 0.1 s
- **Direct-Axis Reactance (Xd​):** from 1.0 pu to 2.5 pu
- **Quadrature-Axis Reactance (Xq​):** from 1.0 pu to 2.5 pu
- **Transient Direct-Axis Reactance (Xd′​):** from 0.2 pu to 0.3 pu
- **Shaft Time Constant (H):** from 2 s to 6 s
- **Turbine Time Constant (T):** from 4 s to 10 s
- **Exciter Voltage Time Constant (Ta):** from 0.02 s to 0.5 s
- **Exciter Voltage Gain (Ka):** from 10 to 500
- **Primary Control Gain (Kp):** from 0.5 to 200
- **Damping:** from 0 to 50

### Excitation Change Analysis (Vs from 1.01 pu to 1.05 pu)
This analysis explores the impact of changing the stator voltage from 1.01 pu to 1.05 pu on both the linear and nonlinear models.

### Mechanical Power Change Analysis (Pm from initial value to 1.25 pu)
It evaluates how increasing mechanical power from its initial value to 1.25 pu, while maintaining the stator voltage at 1.05 pu, affects both the linear and nonlinear models.

### Comparison
The responses of the **linear models** and **nonlinear models** are compared regarding changes in excitation and mechanical power, assessing system stability, time response, and oscillations.

**Note:** The equations for the nonlinear model are based on the provided paper (Eq. (4), Page 8).



### Necessary Equations for the Nonlinear Model

The third-order transient model of a synchronous machine connected to an infinite bus is described by **Equation (4)** of the paper, which represents the dynamics of the rotor angle $(\delta)$, the rotor angular velocity $(\omega)$, and the transient stress in the quadrature axis $(E_q')$. The key equations are as follows:

1. **Dynamic Equations:**
   $
   \dot{\delta} = \Delta\omega
   $

   $
   M \dot{\Delta} \dot{\omega} = P_m - \frac{V E_q' \sin(\delta)}{X_d'} + \frac{V^2 \sin(2\delta)}{2X_q}
   $

   $
   \dot{E_q'} = -\frac{E_q'}{T_d a'} + \frac{V_f \cos(\delta)}{T_d a'} + \frac{E_f}{T_do'}
   $

   Where:

   - $M$ is the moment of inertia of the generator.
   - $X_d'$ is the transient reactance on the d axis.
   - $X_q$ is the reactance on the q axis.
   - $T_d'$ is the transient time constant on the d axis.

2. **Terminal Voltage Equation:**
   The equation for the generator terminal voltage $v_t$ is presented as:
   $
v_t = \sqrt{\left( \frac{ x_3 x_e + {V x_d' \cos(x_2)}}{a} \right)^2 + \left( \frac{V x_q \sin(x_2)}{c} \right)^2}
$
   
   
   where $v_t$ is the desired value of the terminal voltage (see page (8) of the document).

This nonlinear system represents the fundamental dynamics of the synchronous generator, considering the excitation voltage $(V_f)$ and the effects of reactances. These equations are essential to implement the model in the simulation of the system's time response.






In [10]:
# 1. Importación de librerías
import numpy as np
from scipy.integrate import odeint
import plotly.graph_objects as go

# 2. Definición de parámetros de la máquina síncrona
Xd = 1.75  # Reactancia del eje directo (pu)
Xq = 1.75  # Reactancia del eje en cuadratura (pu)
Td0 = 0.055  # Constante de tiempo del eje directo (s)
H = 4.0  # Constante de tiempo del eje (s)
Ka = 255.0  # Ganancia del excitador
Kp = 100.25  # Control primario
T = 7.0  # Tiempo constante de la turbina (s)
Xd_prim = 0.25  # Reactancia transitoria del eje directo (pu)
Damping = 0  # Amortiguamiento (pu)

# 3. Inicialización de variables del sistema
Vs_initial = 1.01  # Tensión inicial (pu)
Eb = 1.01  # Voltaje de la barra infinita (pu)
Pm_initial = 0.5  # Potencia mecánica inicial (pu)
Eq_initial = 0.5  # Tensión de excitación inicial (pu)

# 4. Definición del modelo no lineal
def model_nonlinear(y, t, Vs, Pm):
    """
    Modelo no lineal de la máquina síncrona.

    Parameters:
    y (list): Estado actual del sistema [delta, omega, Eq_prime].
    t (float): Tiempo actual.
    Vs (float): Voltaje del estator.
    Pm (float): Potencia mecánica.

    Returns:
    list: Derivadas del estado [ddelta_dt, domega_dt, dEq_prime_dt].

    """
    delta, omega, Eq_prime = y
    M = 2 * H * 9.81 / 3.14  # Masa equivalente
    a = Xd_prim
    b = Xq - Xd_prim
    c = Xq

    # Ecuaciones del modelo no lineal
    ddelta_dt = omega
    domega_dt = (Pm - (a * Eq_prime * np.sin(delta)) / M +
                  (2 * a * c * (Vs**2) * np.sin(2 * delta)) / (2 * M))
    dEq_prime_dt = (-Eq_prime / Td0 + (Vs * np.cos(delta) / Td0) +
                     (Eq_initial / Td0) - Damping * omega)

    return [ddelta_dt, domega_dt, dEq_prime_dt]

# 5. Definición del modelo lineal
def model_linear(y, t, Vs, Pm):
    """
    Modelo lineal de la máquina síncrona.

    Parameters:
    y (list): Estado actual del sistema [delta, omega, Eq_prime].
    t (float): Tiempo actual.
    Vs (float): Voltaje del estator.
    Pm (float): Potencia mecánica.

    Returns:
    list: Derivadas del estado [ddelta_dt, domega_dt, dEq_prime_dt].

    """
    delta, omega, Eq_prime = y
    M = 2 * H * 9.81 / 3.14  # Masa equivalente
    a = Xd_prim
    b = Xq - Xd_prim
    c = Xq

    # Ecuaciones del modelo lineal
    ddelta_dt = omega
    domega_dt = (Kp * (Pm - (Vs * Eq_prime * np.sin(delta))) / M -
                  Damping * omega)
    dEq_prime_dt = (-Eq_prime / Td0 + (Vs * np.cos(delta) / Td0))

    return [ddelta_dt, domega_dt, dEq_prime_dt]

# 6. Función principal para simular el sistema
def simulate_system():
    """
    Simula la respuesta del sistema para modelos lineal y no lineal,
    considerando cambios en el voltaje del estator y la potencia mecánica.

    """
    # 6.1 Tiempo de simulación
    t = np.linspace(0, 10, 1000)  # 10 segundos

    # 6.2 Condiciones iniciales
    y0 = [0, 0, Eq_initial]  # [delta, omega, Eq_prime]

    # 6.3 Simulación del modelo no lineal con condiciones iniciales
    results_nonlinear_initial = odeint(model_nonlinear, y0, t, args=(Vs_initial, Pm_initial))

    # 6.4 Momentos para cambiar parámetros
    Vs_change_time = 2.5  # Tiempo para el cambio de Vs
    Pm_change_time = 5.0  # Tiempo para el cambio de Pm

    # 6.5 Resultados de la simulación
    results_nonlinear = []
    results_linear = []

    # 6.6 Simulación paso a paso
    for i in range(len(t)):
        # Cambiar Vs de 1.01 a 1.05 en t = 2.5
        if t[i] >= Vs_change_time:
            Vs_current = 1.05
        else:
            Vs_current = Vs_initial

        # Cambiar Pm de 0.5 a 1.25 en t = 5.0
        if t[i] >= Pm_change_time:
            Pm_current = 1.25
        else:
            Pm_current = Pm_initial

        # Simulación del modelo no lineal
        if i == 0:
            results_nonlinear.append(y0)
        else:
            y_last = results_nonlinear[-1]
            y_next = odeint(model_nonlinear, y_last, [t[i-1], t[i]], args=(Vs_current, Pm_current))[1]
            results_nonlinear.append(y_next)

        # Simulación del modelo lineal
        if i == 0:
            results_linear.append(y0)
        else:
            y_last_linear = results_linear[-1]
            y_next_linear = odeint(model_linear, y_last_linear, [t[i-1], t[i]], args=(Vs_current, Pm_current))[1]
            results_linear.append(y_next_linear)

    # 6.7 Convertir resultados a arrays
    results_nonlinear = np.array(results_nonlinear)
    results_linear = np.array(results_linear)

    # 6.8 Graficas
    plot_results(t, results_nonlinear, results_linear)

# 7. Función para graficar resultados
def plot_results(t, results_nonlinear, results_linear):
    """
    Genera las gráficas de los resultados de las simulaciones.

    Parameters:
    t (array): Tiempo de simulación.
    results_nonlinear (array): Resultados de la simulación no lineal.
    results_linear (array): Resultados de la simulación lineal.

    """
    # 7.1 Gráfica de deltas
    fig_delta = go.Figure()

    # 7.2 Graficar delta
    fig_delta.add_trace(go.Scatter(x=t, y=results_nonlinear[:, 0],
                                    mode='lines', name='Modelo No Lineal (delta)',
                                    line=dict(color='blue')))
    fig_delta.add_trace(go.Scatter(x=t, y=results_linear[:, 0],
                                    mode='lines', name='Modelo Lineal (delta)',
                                    line=dict(color='red')))

    # 7.3 Personalizar gráfica de deltas
    fig_delta.update_layout(
        title='Simulación de Delta (Máquina Síncrona)',
        xaxis_title='Tiempo (s)',
        yaxis_title='Delta',
        legend=dict(x=0, y=1),
        template='plotly_dark'
    )

    # 7.4 Mostrar gráfica de deltas
    fig_delta.show()



    # 7.5 Gráfica de omegas
    fig_omega = go.Figure()

    # 7.6 Graficar omega
    fig_omega.add_trace(go.Scatter(x=t, y=results_nonlinear[:, 1],
                                    mode='lines', name='Modelo No Lineal (omega)',
                                    line=dict(color='green')))
    fig_omega.add_trace(go.Scatter(x=t, y=results_linear[:, 1],
                                    mode='lines', name='Modelo Lineal (omega)',
                                    line=dict(color='orange')))

    # 7.7 Personalizar gráfica de omegas
    fig_omega.update_layout(
        title='Simulación de Omega (Máquina Síncrona)',
        xaxis_title='Tiempo (s)',
        yaxis_title='Omega',
        legend=dict(x=0, y=1),
        template='plotly_dark'
    )

    # 7.8 Mostrar gráfica de omegas
    fig_omega.show()

# 8. Ejecución de la simulación
if __name__ == "__main__":
    simulate_system()



#  Code Analysis

The code provides a simulation of the dynamic behavior of a synchronous machine using both linear and nonlinear models. Here are the key points:

## Libraries Used

- It imports `numpy` for numerical calculations, `scipy` for integrating differential equations, and `plotly` for generating interactive plots.

## Clear Parameters

- All physical and control parameters (such as reactances and time constants) are well-defined, facilitating understanding and adjustment.

## Simulation Models

- Two models are implemented: one linear and the other nonlinear, reflecting the machine's dynamics. Both are functions that allow easy integration into the simulation process.

## Simulation and Visualization

- The function `simulate_system()` manages the simulation and allows changes in voltage and mechanical power at specific moments. Visualization of results is done through interactive graphs, enhancing data analysis.

## Observations

### Clear Structure

- The code is well-organized and documented, making it easy to read and maintain.

### Flexibility

- The modular structure allows for easy adjustment of parameters or the addition of new models in the future.

### Realism

- The nonlinear model is more representative of real-world situations, especially in the face of disturbances, which is crucial for stability and performance studies.


## References

Gil-González, W., Garces, A., & Escobar, A. (2019). Applied Mathematical Modelling, 68, 471-486.
